# PDF Extraction

In [31]:
import PyPDF2
import os

# Input PDF file (change this to the path of your PDF)
input_pdf_path = r"C:\Users\abdul\Downloads\form_rec\1.Data 01_50000.pdf"
# Output directory for the split PDFs
output_directory = r"C:\Users\abdul\Downloads\form_rec\output_pdfs"
# Open the input PDF file
# Ensure the output directory exists, or create it

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Open the input PDF file
with open(input_pdf_path, "rb") as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Split the PDF into multiple files
    page_count = len(pdf_reader.pages)
    for i in range(0, page_count, 2):
        pdf_writer = PyPDF2.PdfWriter()
        for j in range(2):
            page_num = i + j
            if page_num < page_count:
                page = pdf_reader.pages[page_num]
                pdf_writer.add_page(page)

        # Output PDF file name (e.g., output_1.pdf, output_2.pdf, etc.)
        output_pdf_path = os.path.join(output_directory, f"output_{i // 2 + 1}.pdf")

        # Write the split PDF to the output file
        with open(output_pdf_path, "wb") as output_pdf_file:
            pdf_writer.write(output_pdf_file)

# OCR

In [24]:
endpoint = "Enter your Azure Endpoint"
api_key = "Enter your API KEY "

In [25]:
import os
import pandas as pd
import json
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
# Create a Document Analysis client
document_analysis_client = DocumentAnalysisClient(endpoint, AzureKeyCredential(api_key))

In [32]:
df=pd.DataFrame()
lenfiles=len(list(os.listdir(r"C:\Users\abdul\Downloads\form_rec\output_pdfs")))

In [27]:
for n in range(lenfiles):
    # Replace with your Azure Form Recognizer endpoint and API key
    pdf_file_path = fr"C:\Users\abdul\Downloads\form_rec\output_pdfs\output_{n}.pdf"
    json_file_path = fr"C:\Users\abdul\Downloads\form_rec\output_json\outputjson_{n}.json"
    # Begin the analysis using the "General document" model
    with open(pdf_file_path, "rb") as pdf_file:
        poller = document_analysis_client.begin_analyze_document("prebuilt-document", document=pdf_file)
        result = poller.result()

    # Convert the document analysis result to a dictionary
    result_dict = result.to_dict()
    # Specify the path where you want to save the JSON file

    # Save the JSON data to the file
    with open(json_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(result_dict, output_file, ensure_ascii=False, indent=2)
    # Open JSON file from local
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
    table_data = []
    for table in data["tables"]:
        for cell in table["cells"]:
            content = cell["content"]
            table_data.append(content)
    num_columns = data["tables"][0]["column_count"]

    # Split the data into rows and columns
    table_rows = [table_data[i:i + num_columns] for i in range(0, len(table_data), num_columns)]

    # Create a DataFrame
    df1= pd.DataFrame(table_rows, columns=[f"Column_{i+1}" for i in range(num_columns)])
    df=pd.concat([df,df1])
    if n==3:
        print("Done ",{n})
        break

In [29]:
df.columns=['Sr.no','Suchna','date',"Name",'place','phone',"Notice"]
# df.to_excel(r"C:\Users\abdul\Downloads\form_rec\Extracted_Data.xlsx",index=False)

In [35]:
df

""
